In [125]:
%load_ext autoreload
%autoreload 2
import sys, os
sys.path.append(os.path.dirname(os.getcwd())) # Include ../SSD in path
import numpy as np
import torch
import matplotlib.pyplot as plt
from vizer.draw import draw_boxes
from tops.config import instantiate, LazyConfig
from ssd import utils

import torchvision
from torch.utils.tensorboard import SummaryWriter

np.random.seed(0)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
%cd ../

/Users/mariu/dev/school/TDT4265/project/computer-vision-and-deep-learning/SSD


In [126]:
run = "task2_2"
config_path = f"configs/{run}.py"
cfg = LazyConfig.load(config_path)

Saving SSD outputs to: outputs/


In [127]:
dataset_to_visualize = "train" # or "val"

# THIS NEEDS TO MATCH BATCH USED WHEN TRAINING, else dataloader goes fucking crazy
cfg.train.batch_size = 1

if dataset_to_visualize == "train":
    # Remove GroundTruthBoxesToAnchors transform
    if cfg.data_train.dataset._target_ == torch.utils.data.ConcatDataset:
        for dataset in cfg.data_train.dataset.datasets:
            dataset.transform.transforms = dataset.transform.transforms[:-1]
    else:
        cfg.data_train.dataset.transform.transforms = cfg.data_train.dataset.transform.transforms[:-1]
    dataset = instantiate(cfg.data_train.dataloader)
    gpu_transform = instantiate(cfg.data_train.gpu_transform)
else:
    cfg.data_val.dataloader.collate_fn = utils.batch_collate
    dataset = instantiate(cfg.data_val.dataloader)
    gpu_transform = instantiate(cfg.data_val.gpu_transform)

# Assumes that the first GPU transform is Normalize
# If it fails, just change the index from 0.
# image_mean = torch.tensor(cfg.data_train.gpu_transform.transforms[0].mean).view(1, 3, 1, 1)
# image_std = torch.tensor(cfg.data_train.gpu_transform.transforms[0].std).view(1, 3, 1, 1)
model = instantiate(cfg.model)

In [128]:
sample = next(iter(dataset))
sample = gpu_transform(sample)

In [121]:
# from torch.jit import _trace

# module = _trace(model.forward(sample["image"]), sample["image"])

In [113]:
# Create dummy input
dummy_input = torch.rand(1, 3, 128, 1024)
shape = torch.Size((1, 3, 128, 1024))
# dummy_input = torch.cuda.FloatTensor(shape)
torch.randn(shape, out=dummy_input)
print(dummy_input.shape)
print(type(dummy_input))

torch.Size([1, 3, 128, 1024])
<class 'torch.Tensor'>


In [129]:
tb = SummaryWriter(f'notebooks/runs/{run}')
model = instantiate(cfg.model)

image = sample["image"]

tb.add_graph(model, image)
tb.close()

In [135]:
# Test Pytorch SSD300 implementation
# Might have to use: map_location=torch.device('cpu')
ssd_model = torch.hub.load('NVIDIA/DeepLearningExamples:torchhub', 'nvidia_ssd')
utils_ = torch.hub.load('NVIDIA/DeepLearningExamples:torchhub', 'nvidia_ssd_processing_utils')

In [ ]:
ssd_model.to('cuda')
ssd_model.eval()

In [ ]:
uris = [
    'http://images.cocodataset.org/val2017/000000397133.jpg',
    'http://images.cocodataset.org/val2017/000000037777.jpg',
    'http://images.cocodataset.org/val2017/000000252219.jpg'
]

In [ ]:
inputs = [utils_.prepare_input(uri) for uri in uris]
tensor = utils_.prepare_tensor(inputs)

In [ ]:
tb.add_graph(ssd_model, tensor[0])
tb.close()

In [ ]:
with torch.no_grad():
    detections_batch = ssd_model(tensor)

In [ ]:
results_per_input = utils_.decode_results(detections_batch)
best_results_per_input = [utils_.pick_best(results, 0.40) for results in results_per_input]

In [ ]:
classes_to_labels = utils_.get_coco_object_dictionary()

In [ ]:
from matplotlib import pyplot as plt
import matplotlib.patches as patches

for image_idx in range(len(best_results_per_input)):
    fig, ax = plt.subplots(1)
    # Show original, denormalized image...
    image = inputs[image_idx] / 2 + 0.5
    ax.imshow(image)
    # ...with detections
    bboxes, classes, confidences = best_results_per_input[image_idx]
    for idx in range(len(bboxes)):
        left, bot, right, top = bboxes[idx]
        x, y, w, h = [val * 300 for val in [left, bot, right - left, top - bot]]
        rect = patches.Rectangle((x, y), w, h, linewidth=1, edgecolor='r', facecolor='none')
        ax.add_patch(rect)
        ax.text(x, y, "{} {:.0f}%".format(classes_to_labels[classes[idx] - 1], confidences[idx]*100), bbox=dict(facecolor='white', alpha=0.5))
plt.show()